# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [2]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [3]:
# Your code here; create the connection
conn = sqlite3.Connection("data/data2.sqlite")

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [16]:
# Your code here
pd.read_sql('''
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM
    customers
WHERE customerNumber IN
(
    SELECT
       customerNumber
    FROM
        orders
    WHERE
        orderDate = '2003-01-31'
)
    
''', conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [4]:
# Your code here
pd.read_sql('''
SELECT
    productName,
    SUM(quantityOrdered) as total_number_of_orders
FROM
    products
JOIN
    orderdetails
        USING(productCode)
GROUP BY
    productName
''', conn)

,productName,total_number_of_orders
0,18th Century Vintage Horse Carriage,907
1,18th century schooner,1011
2,1900s Vintage Bi-Plane,940
3,1900s Vintage Tri-Plane,1009
4,1903 Ford Model A,883
...,...,...
104,The Mayflower,898
105,The Queen Mary,896
106,The Schooner Bluenose,934
107,The Titanic,952


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [15]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
pd.read_sql('''
SELECT
    productName,
    COUNT(customerName) as customer_amount
FROM
    customers
JOIN
    orders
        USING(customerNumber)
JOIN
    orderdetails
        USING(orderNumber)
JOIN
    products
        USING(productCode)
GROUP BY
    productName
ORDER BY
    customer_amount ASC
''', conn)

,productName,customer_amount
0,1952 Citroen-15CV,24
1,1957 Ford Thunderbird,24
2,1911 Ford Town Car,25
3,1917 Grand Touring Sedan,25
4,1932 Alfa Romeo 8C2300 Spider Sport,25
...,...,...
104,Diamond T620 Semi-Skirted Tanker,28
105,F/A 18 Hornet 1/72,28
106,HMS Bounty,28
107,P-51-D Mustang,28


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [4]:
# Your code here
pd.read_sql('''
SELECT
    DISTINCT employeeNumber,
    officeCode,
    o.city,
    firstName,
    lastName
FROM 
    employees e
JOIN 
    offices o
        USING(officeCode)
JOIN 
    customers c
        ON 
            e.employeeNumber = c.salesRepEmployeeNumber
JOIN 
    orders
        USING(customerNumber)
JOIN 
    orderdetails
        USING(orderNumber)
WHERE productCode IN 
(
    SELECT 
        productCode
    FROM 
        products
    JOIN 
        orderdetails
            USING(productCode)
    JOIN 
        orders
            USING(orderNumber)
    GROUP BY 
        productCode
    HAVING 
        COUNT(DISTINCT customerNumber) < 20
)

''', conn)

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1501,7,London,Larry,Bott
2,1337,4,Paris,Loui,Bondur
3,1166,1,San Francisco,Leslie,Thompson
4,1286,3,NYC,Foon Yue,Tseng
5,1612,6,Sydney,Peter,Marsh
6,1611,6,Sydney,Andy,Fixter
7,1401,4,Paris,Pamela,Castillo
8,1621,5,Tokyo,Mami,Nishi
9,1323,3,NYC,George,Vanauf


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [5]:
# Your code here
pd.read_sql('''
SELECT
    employeeNumber,
    firstName,
    lastName,
    COUNT(customerNumber) AS number_of_customers
FROM 
    employees e
JOIN 
    customers c
        ON 
            e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY 
    employeeNumber
HAVING 
    AVG(creditLimit) > 15000
''', conn)   

,employeeNumber,firstName,lastName,number_of_customers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!